# 第9章: PHPの基礎をマスターしよう

## 学習目標
- PHPの高度な基礎機能を理解する
- クラスとオブジェクト指向の基礎を学ぶ
- 例外処理を使いこなす
- 名前空間を理解する
- 日付・時刻の扱いをマスターする

---
## 1. クラスとオブジェクト指向の基礎

クラスは、データと処理をまとめた設計図です。

### 🔍 クラスの基本構造

```php
class クラス名 {
    // プロパティ（変数）
    public $property;
    
    // メソッド（関数）
    public function method() {
        // 処理
    }
}
```

In [ ]:
<?php

/**
 * Userクラスの定義
 */
class User {
    // プロパティ
    public $name;
    public $email;
    private $password;
    
    // コンストラクタ（インスタンス生成時に実行）
    public function __construct(string $name, string $email, string $password) {
        $this->name = $name;
        $this->email = $email;
        $this->password = $password;
    }
    
    // メソッド
    public function getInfo(): string {
        return "名前: {$this->name}, メール: {$this->email}";
    }
    
    public function verifyPassword(string $input): bool {
        return $this->password === $input;
    }
    
    // セッター
    public function setEmail(string $email): void {
        if (filter_var($email, FILTER_VALIDATE_EMAIL)) {
            $this->email = $email;
        }
    }
}

// インスタンス生成
$user = new User('山田太郎', 'yamada@example.com', 'password123');

echo $user->getInfo() . "\n";

var_dump($user->verifyPassword('password123'));  // true
var_dump($user->verifyPassword('wrong'));       // false

$user->setEmail('newemail@example.com');
echo "新しいメール: {$user->email}\n";

?>

---
## 2. アクセス修飾子

| 修飾子 | 意味 | アクセス範囲 |
|--------|------|-------------|
| public | 公開 | どこからでもアクセス可能 |
| protected | 保護 | クラス内と継承クラスからのみ |
| private | 非公開 | クラス内のみ |

In [ ]:
<?php

class BankAccount {
    private $balance;
    public $owner;
    
    public function __construct(string $owner, int $initialBalance = 0) {
        $this->owner = $owner;
        $this->balance = $initialBalance;
    }
    
    public function deposit(int $amount): void {
        if ($amount > 0) {
            $this->balance += $amount;
        }
    }
    
    public function withdraw(int $amount): bool {
        if ($amount > 0 && $this->balance >= $amount) {
            $this->balance -= $amount;
            return true;
        }
        return false;
    }
    
    public function getBalance(): int {
        return $this->balance;
    }
}

$account = new BankAccount('鈴木一郎', 1000);

echo "口座名義: {$account->owner}\n";
echo "残高: {$account->getBalance()}円\n";

$account->deposit(500);
echo "入金後の残高: {$account->getBalance()}円\n";

$success = $account->withdraw(300);
echo "出金" . ($success ? "成功" : "失敗") . "\n";
echo "最終残高: {$account->getBalance()}円\n";

// 直接アクセスはできない
// $account->balance = 99999;  // エラー！

?>

---
## 3. 継承

既存のクラスをベースに新しいクラスを作成できます。

In [ ]:
<?php

// 親クラス
class Animal {
    protected $name;
    
    public function __construct(string $name) {
        $this->name = $name;
    }
    
    public function eat(): void {
        echo "{$this->name}は食事をしている\n";
    }
    
    public function sleep(): void {
        echo "{$this->name}は眠っている\n";
    }
}

// 子クラス
class Dog extends Animal {
    public function bark(): void {
        echo "{$this->name}：ワンワン！\n";
    }
    
    // オーバーライド
    public function eat(): void {
        echo "{$this->name}はドッグフードを食べている\n";
    }
}

class Cat extends Animal {
    public function meow(): void {
        echo "{$this->name}：ニャー！\n";
    }
}

$dog = new Dog('ポチ');
$dog->eat();    // オーバーライドされたメソッド
$dog->sleep();  // 親クラスのメソッド
$dog->bark();   // 子クラス独自のメソッド

$cat = new Cat('タマ');
$cat->eat();
$cat->meow();

?>

---
## 4. 例外処理

エラーが発生した場合の処理を制御できます。

In [ ]:
<?php

// 基本的な例外処理
function divide(int $a, int $b): float {
    if ($b === 0) {
        throw new Exception("0で除算することはできません");
    }
    return $a / $b;
}

try {
    echo divide(10, 2) . "\n";
    echo divide(10, 0) . "\n";
} catch (Exception $e) {
    echo "エラー: " . $e->getMessage() . "\n";
    echo "ファイル: " . $e->getFile() . "\n";
    echo "行番号: " . $e->getLine() . "\n";
}

echo "処理が継続されました\n";

?>

### カスタム例外クラス

In [ ]:
<?php

// カスタム例外クラス
class ValidationException extends Exception {}
class DatabaseException extends Exception {}

class UserRegistration {
    public function register(string $email, string $password): void {
        $this->validateEmail($email);
        $this->validatePassword($password);
        $this->saveToDatabase($email, $password);
        echo "登録完了！\n";
    }
    
    private function validateEmail(string $email): void {
        if (!filter_var($email, FILTER_VALIDATE_EMAIL)) {
            throw new ValidationException("無効なメールアドレスです");
        }
    }
    
    private function validatePassword(string $password): void {
        if (strlen($password) < 8) {
            throw new ValidationException("パスワードは8文字以上である必要があります");
        }
    }
    
    private function saveToDatabase(string $email, string $password): void {
        // デモ用: 50%の確率で失敗
        if (rand(1, 2) === 1) {
            throw new DatabaseException("データベースエラーが発生しました");
        }
    }
}

$registration = new UserRegistration();

try {
    $registration->register('user@example.com', 'password123');
} catch (ValidationException $e) {
    echo "入力エラー: " . $e->getMessage() . "\n";
} catch (DatabaseException $e) {
    echo "システムエラー: " . $e->getMessage() . "\n";
} catch (Exception $e) {
    echo "予期しないエラー: " . $e->getMessage() . "\n";
}

?>

---
## 5. 名前空間

クラス名の衝突を防ぐために名前空間を使用します。

In [ ]:
<?php

namespace App\Models;

class User {
    public function __construct() {
        echo "App\\Models\\Userが生成されました\n";
    }
}

namespace App\Views;

class User {
    public function __construct() {
        echo "App\\Views\\Userが生成されました\n";
    }
}

namespace {
    // 完全修飾名で使用
    $modelUser = new \App\Models\User();
    $viewUser = new \App\Views\User();
    
    // useでエイリアスを作成
    use App\Models\User as UserModel;
    use App\Views\User as UserView;
    
    $user1 = new UserModel();
    $user2 = new UserView();
}

?>

---
## 6. 日付・時刻の操作

日付と時刻の操作はPHPで頻繁に使われます。

In [ ]:
<?php

// 現在の日時
echo "現在: " . date('Y-m-d H:i:s') . "\n";

// タイムスタンプから日付
$timestamp = time();
echo "タイムスタンプ: {$timestamp}\n";
echo "日付: " . date('Y年m月d日', $timestamp) . "\n";

// 特定の日時の作成
$date = new DateTime('2026-02-04 15:30:00');
echo "DateTime: " . $date->format('Y-m-d H:i:s') . "\n";

// 日付の加減算
$tomorrow = new DateTime('tomorrow');
echo "明日: " . $tomorrow->format('Y-m-d') . "\n";

$nextWeek = new DateTime('+1 week');
echo "1週間後: " . $nextWeek->format('Y-m-d') . "\n";

// 期間の計算
$date1 = new DateTime('2026-01-01');
$date2 = new DateTime('2026-12-31');
$interval = $date1->diff($date2);
echo "期間: " . $interval->format('%a日') . "\n";

// 曜日の取得
$weekdays = ['日', '月', '火', '水', '木', '金', '土'];
$weekday = $weekdays[date('w')];
echo "今日は" . $weekday . "曜日\n";

?>

### 実践的な日付操作

In [ ]:
<?php

/**
 * 営業日計算クラス
 */
class BusinessDayCalculator {
    private $holidays;
    
    public function __construct(array $holidays = []) {
        $this->holidays = $holidays;
    }
    
    /**
     * 営業日を加算
     */
    public function addBusinessDays(string $date, int $days): string {
        $current = new DateTime($date);
        $addedDays = 0;
        
        while ($addedDays < $days) {
            $current->modify('+1 day');
            
            if ($this->isBusinessDay($current)) {
                $addedDays++;
            }
        }
        
        return $current->format('Y-m-d');
    }
    
    /**
     * 営業日かどうか判定
     */
    private function isBusinessDay(DateTime $date): bool {
        // 土日チェック
        $weekday = (int)$date->format('w');
        if ($weekday === 0 || $weekday === 6) {
            return false;
        }
        
        // 祝日チェック
        $dateStr = $date->format('Y-m-d');
        if (in_array($dateStr, $this->holidays)) {
            return false;
        }
        
        return true;
    }
}

// 使用例
$holidays = [
    '2026-01-01',  // 元日
    '2026-02-11',  // 建国記念の日
    '2026-02-23',  // 天皇誕生日
];

$calculator = new BusinessDayCalculator($holidays);

$startDate = '2026-02-02';  // 月曜日
$deliveryDate = $calculator->addBusinessDays($startDate, 5);

echo "開始日: {$startDate}\n";
echo "納品日（5営業日後）: {$deliveryDate}\n";

?>

---
## 7. ファイル操作

ファイルの読み書きは実務で頻繁に使われます。

In [ ]:
<?php

// ファイル書き込み
$filename = __DIR__ . '/test_log.txt';
$content = "ログ出力テスト\n";
$content .= "日時: " . date('Y-m-d H:i:s') . "\n";

// 上書きモード
file_put_contents($filename, $content);
echo "ファイルを作成しました: {$filename}\n";

// 追記モード
file_put_contents($filename, "追加のログ\n", FILE_APPEND);

// ファイル読み込み
$readContent = file_get_contents($filename);
echo "\nファイル内容:\n{$readContent}\n";

// 配列として読み込み（行ごと）
$lines = file($filename, FILE_IGNORE_NEW_LINES);
echo "\n行数: " . count($lines) . "\n";

// ファイル存在確認
if (file_exists($filename)) {
    echo "サイズ: " . filesize($filename) . "バイト\n";
    echo "更新日時: " . date('Y-m-d H:i:s', filemtime($filename)) . "\n";
}

// ファイル削除
if (unlink($filename)) {
    echo "\nファイルを削除しました\n";
}

?>

---
## 8. 実践的なクラス設計

これまでの知識を組み合わせて、実用的なクラスを作成しましょう。

In [ ]:
<?php

/**
 * シンプルなロガークラス
 */
class Logger {
    private $logFile;
    private $minLevel;
    
    const DEBUG = 1;
    const INFO = 2;
    const WARNING = 3;
    const ERROR = 4;
    
    private static $levelNames = [
        self::DEBUG => 'DEBUG',
        self::INFO => 'INFO',
        self::WARNING => 'WARNING',
        self::ERROR => 'ERROR'
    ];
    
    public function __construct(string $logFile, int $minLevel = self::INFO) {
        $this->logFile = $logFile;
        $this->minLevel = $minLevel;
        
        // ディレクトリがなければ作成
        $logDir = dirname($logFile);
        if (!is_dir($logDir)) {
            mkdir($logDir, 0755, true);
        }
    }
    
    public function log(string $message, int $level = self::INFO): void {
        if ($level < $this->minLevel) {
            return;
        }
        
        $timestamp = date('Y-m-d H:i:s');
        $levelName = self::$levelNames[$level];
        $logMessage = "[{$timestamp}] [{$levelName}] {$message}\n";
        
        file_put_contents($this->logFile, $logMessage, FILE_APPEND);
    }
    
    public function debug(string $message): void {
        $this->log($message, self::DEBUG);
    }
    
    public function info(string $message): void {
        $this->log($message, self::INFO);
    }
    
    public function warning(string $message): void {
        $this->log($message, self::WARNING);
    }
    
    public function error(string $message): void {
        $this->log($message, self::ERROR);
    }
    
    /**
     * ログをクリア
     */
    public function clear(): void {
        file_put_contents($this->logFile, '');
    }
    
    /**
     * ログを取得
     */
    public function getLogs(): array {
        if (!file_exists($this->logFile)) {
            return [];
        }
        return file($this->logFile, FILE_IGNORE_NEW_LINES);
    }
}

// 使用例
$logger = new Logger(__DIR__ . '/logs/app.log', Logger::DEBUG);

$logger->debug('デバッグ情報');
$logger->info('処理開始');
$logger->warning('警告メッセージ');
$logger->error('エラー発生');

echo "ログファイル: " . __DIR__ . '/logs/app.log\n';
echo "\nログ内容:\n";
foreach ($logger->getLogs() as $log) {
    echo $log . "\n";
}

?>

---
## 📝 実践問題

### 問題1: 製品クラス

製品を管理するクラスを作成してください。

- プロパティ: 名前、価格、在庫数
- メソッド: 在庫追加、販売、在庫確認
- 在庫不足時の例外処理

### 問題2: 日付バリデーション

以下のチェックを行う関数を作成してください。

- 日付形式が正しいか
- 過去の日付でないか
- 営業日かどうか

### 問題3: 設定ファイル管理クラス

YAMLまたはJSON形式の設定ファイルを読み書きするクラスを作成してください。

### 問題4: シンプルなMVC構造

以下のクラスを作成してください。

- Model: データ管理
- View: 表示
- Controller: 処理の制御

---
## まとめ

### ✅ ポイントチェック

- [ ] クラスとオブジェクトの基本を理解した
- [ ] カプセル化（アクセス修飾子）を使える
- [ ] 継承の概念を理解した
- [ ] 例外処理を実装できる
- [ ] 名前空間を使いこなせる
- [ ] 日付・時刻の操作ができる
- [ ] ファイル操作ができる

### 🎯 次のステップ

次は「配列の操作をマスターしよう」で、配列の高度な操作を学びましょう！

---

**関連リソース**
- [PHP: クラスとオブジェクト](https://www.php.net/manual/ja/language.oop5.php)
- [PHP: 例外](https://www.php.net/manual/ja/language.exceptions.php)
- [PHP: DateTime](https://www.php.net/manual/ja/class.datetime.php)